# Programmatic Tuning with VAMOS

While the `vamos-tune` CLI is convenient, you often want to integrate tuning directly into your Python scripts or notebooks.
This notebook demonstrates how to use the **RacingTuner** API programmatically to optimize NSGA-II on the **ZDT1** problem.

## Steps
1.  **Define Parameter Space**: Using `build_nsgaii_config_space`.
2.  **Configure Scenario**: Budget, parallel jobs, etc.
3.  **Run Tuner**: Execute the race.
4.  **Validate**: Run the best configuration found.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# VAMOS Imports
from vamos.api import optimize, OptimizeConfig
from vamos.engine.algorithm.config import NSGAIIConfig
from vamos.foundation.problem.registry import make_problem_selection
from vamos.foundation.metrics.hypervolume import compute_hypervolume
from vamos.foundation.core.experiment_config import ExperimentConfig
from vamos.foundation.core.runner import run_single

# Tuning Imports
from vamos.engine.tuning.racing.core import RacingTuner
from vamos.engine.tuning.racing.scenario import Scenario
from vamos.engine.tuning.racing.tuning_task import TuningTask, Instance, EvalContext
from vamos.engine.tuning.racing.bridge import build_nsgaii_config_space, config_from_assignment

## 1. Setup Tuning Task
We utilize `build_nsgaii_config_space` to get the standard search space for NSGA-II (pop_size, crossover/mutation probs, etc).

In [ ]:
# 1. Parameter Space
param_space = build_nsgaii_config_space()

# 2. Instances & Seeds
# We tune on ZDT1 (30 vars)
instances = [Instance(name="zdt1", n_var=30, kwargs={})]
seeds = [100, 101, 102, 103, 104] # 5 training seeds

# 3. Task Definition
task = TuningTask(
    name="tune_nsgaii_zdt1",
    param_space=param_space,
    instances=instances,
    seeds=seeds,
    aggregator=lambda scores: np.mean(scores),
    budget_per_run=2000, # Evaluations per run during tuning
    maximize=True # We maximize Hypervolume
)

# 4. Scenario (The Racing Strategy)
scenario = Scenario(
    max_experiments=5000,   # Total tuning budget (small for demo)
    n_jobs=2,               # Parallel jobs
    verbose=True,
    elimination_fraction=0.3
)

print("Tuning Task Set Up:")
print(f"  Param Space Size: ~{len(param_space.params)} params")
print(f"  Budget: {scenario.max_experiments} evals")

## 2. Define Evaluation Function
The tuner calls this function. It receives a `config_dict` and must return a scalar score (Hypervolume).

In [ ]:
def eval_zdt1(config_dict, ctx: EvalContext) -> float:
    try:
        # Convert dict to AlgorithmConfig object
        # Note: pop_size might be missing if fixed, inject if needed
        if "pop_size" not in config_dict:
            config_dict["pop_size"] = 100
            
        cfg = config_from_assignment("nsgaii", config_dict)
        
        # Setup Experiment
        selection = make_problem_selection("zdt1", n_var=ctx.instance.n_var)
        exp_cfg = ExperimentConfig(
            population_size=cfg.population_size, 
            max_evaluations=ctx.budget, 
            seed=ctx.seed
        )
        
        # Run Single Evaluation
        result = run_single(
            engine="numpy",
            algorithm="nsgaii",
            problem_selection=selection,
            experiment_config=exp_cfg,
            algorithm_config=cfg,
            verbose=False
        )
        
        # Calculate HV
        if result.F is None or len(result.F) == 0:
            return 0.0
        
        ref_point = np.array([1.1, 1.1])
        return compute_hypervolume(result.F, ref_point)
        
    except Exception as e:
        print(f"Eval failed: {e}")
        return 0.0

## 3. Run Racing Tuner
This starts the race!

In [ ]:
tuner = RacingTuner(task=task, scenario=scenario, seed=42)

print("Starting Race...")
best_config, history = tuner.run(eval_zdt1)

print("\n--- Tuning Complete ---")
print("Best Configuration found:")
for k, v in best_config.items():
    print(f"  {k}: {v}")

## 4. Validate Best Configuration
We compare the tuned configuration against a default configuration on a fresh seed.

In [ ]:
# Convert best dict to Config object
if "pop_size" not in best_config: best_config["pop_size"] = 100
tuned_cfg = config_from_assignment("nsgaii", best_config)

# Default Config
default_cfg = NSGAIIConfig().pop_size(100).crossover("sbx", prob=0.9).mutation("pm", prob=0.1).selection("tournament").fixed()

def evaluate_final(cfg, name):
    p = make_problem_selection("zdt1").instantiate()
    res = optimize(
        OptimizeConfig(
            problem=p,
            algorithm="nsgaii",
            algorithm_config=cfg,
            termination=("n_eval", 5000),
            seed=999 # Test seed
        )
    )
    hv = compute_hypervolume(res.F, np.array([1.1, 1.1]))
    print(f"{name} HV: {hv:.4f}")
    return hv

print("\nValidation Result:")
hv_tuned = evaluate_final(tuned_cfg, "Tuned")
hv_default = evaluate_final(default_cfg, "Default")